# Objective
Narrow down the set of job postings to those that are most similar to our resume in preparation for further analysis.

# Prepare corpus 

1. Import and remove the resume text of escape sequences.
2. Import the job posting dataset and clean the job requirement column.
3. Combine the job requirement column and the resume text to create a text corpus. 

In [94]:
import pathlib
import pandas as pd
import numpy as np

cur_path = pathlib.Path()
data_path = cur_path.joinpath('output/data.csv')
resume_path = cur_path.joinpath('data/LiveProject Resume.txt')

In [95]:
with open(resume_path) as r:
    resume = r.read()

import re
resume = re.sub('\\t|\\n|\\ufeff|ㅡ', ' ', resume) 
resume

' Good Student Data Scientist      Good Student 123 Fake Street Some City, QT 12345 123.456.7890 no_reply@fakesite.com    Skills      Python, Pandas, machine learning, natural language processing    Experience      Manning / Data Analyst Oct 2019 - PRESENT,  REMOTE Analyzed and visualized vast amounts of data using Pandas, Python, and Matplotlib.    Education      Berkeley / B.S. Mathematics August 2015 - May 2019,  BERKELEY, CA Graduated summa cum laude.     Awards      Tau Beta Pi Honors Society '

In [96]:
df = pd.read_csv(data_path)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   html      754 non-null    object
 1   position  754 non-null    object
 2   location  754 non-null    object
 3   company   110 non-null    object
 4   jobreq    753 non-null    object
dtypes: object(5)
memory usage: 29.6+ KB
None


In [99]:
# remove observation with null value for 'jobreq'
df.dropna(axis = 0, subset = ['jobreq'], inplace = True)
df.reset_index(inplace = True, drop = True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   html      753 non-null    object
 1   position  753 non-null    object
 2   location  753 non-null    object
 3   company   110 non-null    object
 4   jobreq    753 non-null    object
dtypes: object(5)
memory usage: 29.5+ KB
None


In [100]:
"""
Attach resume to df['jobreq'] and name the pd.Series 'corpus'.
The last entry is the resume text.
"""
corpus = df['jobreq'].append(pd.Series(resume)).reset_index(drop = True)
print(corpus)

0      Quantitative Analyst (State Street Bank and Tr...
1      Help senior members of the team to explore, de...
2      A Bachelor or Masters Degree in a highly quant...
3      Join a small team creating a proprietary NLU s...
4      Degree: Bachelor’s degree in business analytic...
                             ...                        
749    Applicable degree(s): Computer Information Sys...
750    Experience with Python Knowledge of some of th...
751    Independently develop advanced analytics and p...
752    Experienced in handling large data sets using ...
753     Good Student Data Scientist      Good Student...
Length: 754, dtype: object


# Compute document frequency of words appearing in the resume

Compute what proportion of job postings share the words that appeared in the resume text. 

1. Set the parameters of `sklearn.TfidfVectorizer` to 
    - stop commonly used English words (`stop_words = "english"`),
    - pick up words that are shared in at least 4 job postings but not appearing in more than 80% of the job postings (`min_df = 4, max_df = 0.8`),
    - and detect unigrams and bigrams (`ngram_range = (1, 2)`),
    - with ivnerse df enabled (`use_idf = True`).
1. Obtain inverse document frequenciese (idf) of the words used in the resume. 
2. Rank the words from the smallest to largest idf where smaller idf means the word is shared across large number of job postings and vice versa.

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer 

vectorizer = TfidfVectorizer(stop_words = "english", min_df = 4, max_df = 0.8, ngram_range = (1, 2)) 
tfidf_matrix = vectorizer.fit_transform(corpus) 

tfidf_np_matrix = tfidf_matrix.toarray() # convert to numpy array
print(tfidf_np_matrix)
print(tfidf_np_matrix.shape)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(754, 8305)


In [89]:
from collections import namedtuple

feature_extraction_result = namedtuple('corpus_features', 
                            ['csr_matrix', 'np_matrix', 'features', 'params'])

def resume_words_df():
    
    # Fit vectorizer
    vectorizer = TfidfVectorizer(stop_words = "english", min_df = 4, max_df = 0.8, 
                                 ngram_range = (1, 2), use_idf = True) 
    tfidf_matrix = vectorizer.fit_transform(corpus) 
    
    tfidf_np_matrix = tfidf_matrix.toarray()
    print("{0} documents and {1} features".format(*tfidf_np_matrix.shape))
    
    
    # Get the words in the resume
    words = vectorizer.get_feature_names()
    
    resume_vector = tfidf_np_matrix[-1] # the last document/row is the resume 
    nonzero_indices = np.flatnonzero(resume_vector)
    resume_words = [words[i] for i in nonzero_indices]
    print("Words in the resume:\n {}".format(resume_words))
    
    
    resume_df = {
        "words": resume_words,
        "tf-idf": resume_vector[nonzero_indices],
        "idf": vectorizer.idf_[nonzero_indices]
    }
    print(pd.DataFrame(resume_df).sort_values('tf-idf', ascending = True))
    
    return feature_extraction_result(csr_matrix = tfidf_matrix,
                          np_matrix = tfidf_np_matrix, 
                           features = words, params = vectorizer.get_params())

corpus_results = resume_words_df()   

754 documents and 8305 features
Words in the resume:
 ['2015', '2019', 'amounts', 'amounts data', 'analyst', 'analyzed', 'august', 'berkeley', 'ca', 'city', 'com', 'data analyst', 'data scientist', 'data using', 'education', 'good', 'language', 'language processing', 'learning', 'learning natural', 'machine', 'machine learning', 'mathematics', 'matplotlib', 'natural', 'natural language', 'pandas', 'pi', 'present', 'processing', 'processing experience', 'python', 'python matplotlib', 'python pandas', 'remote', 'scientist', 'skills', 'skills python', 'street', 'student', 'using', 'vast', 'vast amounts']
                    words    tf-idf       idf
36                 skills  0.041270  1.350074
18               learning  0.043909  1.436402
20                machine  0.045848  1.499849
21       machine learning  0.046049  1.506420
40                  using  0.050527  1.652908
22            mathematics  0.060350  1.974229
16               language  0.066639  2.179980
29             processi

# Compute similarity between job postings and resume

I want to know which job posting is most similar to the resume.

1. Compute cosine similarity between `resume_vector` and `tfidf_np_matrix` with `sklearn.cosine_similarity` where larger value means higher similarity. 
3. Rank job postings from largest to smallest cosine and identify the top 10 job postings as the most similar job postings to our resume. 


In [90]:
from sklearn.metrics.pairwise import cosine_similarity

cosines =  cosine_similarity(corpus_results.csr_matrix, corpus_results.csr_matrix[-1])
print("Cosine similarity of the document with the resume:\n {}".format(cosines))

Cosine similarity of the document with the resume:
 [[7.70543969e-02]
 [2.20957618e-02]
 [2.88940811e-02]
 [5.73653045e-02]
 [1.20006243e-03]
 [3.22765968e-02]
 [1.23749580e-02]
 [2.01166393e-02]
 [1.00612441e-02]
 [1.70852095e-02]
 [1.15410672e-02]
 [1.87172132e-02]
 [0.00000000e+00]
 [1.29208739e-02]
 [1.23197716e-02]
 [2.21875893e-02]
 [1.75037654e-02]
 [4.95756045e-03]
 [1.86904024e-02]
 [9.60793087e-03]
 [4.47603951e-02]
 [4.16510590e-02]
 [6.25255795e-03]
 [8.27446804e-02]
 [3.57619744e-02]
 [5.62390718e-03]
 [4.99634315e-02]
 [8.87546812e-02]
 [1.53551812e-02]
 [3.36359935e-03]
 [3.79537258e-02]
 [1.59438786e-02]
 [1.16182776e-02]
 [2.06866070e-02]
 [2.67607722e-02]
 [1.35904021e-02]
 [8.09604960e-03]
 [3.14733361e-02]
 [3.49645779e-02]
 [5.56740825e-02]
 [6.02485007e-03]
 [3.50948856e-02]
 [9.44247398e-03]
 [2.05166649e-02]
 [1.11890901e-02]
 [7.71451091e-02]
 [3.83244595e-02]
 [6.21776635e-03]
 [5.06255810e-03]
 [1.21123851e-02]
 [3.49067025e-02]
 [2.34075248e-02]
 [3.54094690

In [101]:
ranking_indice = np.argsort(cosines_percentile)
ten_most_similar_ranking_indice = ranking_indice[::-1][1:10] # index 0 is the resume itself

In [104]:
print("Top 10 job postings most similar to the resume:\n {}".format(df.iloc[ten_most_similar_ranking_indice]))

Top 10 job postings most similar to the resume:
                                                   html  \
352  data/html_job_postings/4b13ffbd35f56cf5_fccid....   
360  data/html_job_postings/17a320fb30c7c7ab_fccid....   
491  data/html_job_postings/72435f49f9f550ef_fccid....   
251  data/html_job_postings/ef7046b5961f249e_fccid....   
27   data/html_job_postings/7ba056b7c3fbf8e7_fccid....   
257  data/html_job_postings/242bbe38bd12e721_fccid....   
176  data/html_job_postings/88088691f91338fc_fccid....   
204  data/html_job_postings/7b62436d86347e82_fccid....   
188  data/html_job_postings/551b276663e03b55_fccid....   

                                              position  \
352  machine learning / data scientist internship (...   
360               machine learning postdoctoral fellow   
491                                       data analyst   
251       data science summer institute student intern   
27                                      data scientist   
257                   